## Data Preparation

In this notebook we document the steps taken to prepare data for ML model training.

Further preprocessing (e.g. imputing/encoding) may be required; if this is the case, handle it as part of a preprocessing pipeline before feeding data to a ML model.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
import pandas as pd
from datetime import datetime, timedelta
import warnings

# Local
PATH_TO_SRC = Path('../src').resolve()
sys.path.append(PATH_TO_SRC.as_posix())
import lib_data_prep  # nopep8

# settings
PATH_TO_DATA = Path('../data').resolve()

warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

## Data preparation summary

Only data from `hubway_trip.csv` and `weather.csv` are used. Refer to [`src/lib_data_prep.py`](../src/lib_data_prep.py) for more details on the steps performed.

> Note: Data of hubs location could be used to generate fetures such as *closest_hub_distance*, or *number_of_hubs_within_X_km*, which could all correlated to the average trip duration from and to a given hub. However, these are not built for this POC, as we do not have a log of when a given hub was built/enabled. Should it be neeed, however, this info can retrieved in a second interation by looking at trips starting/ending hubs from the `hubway_trip.csv`.

In [3]:
lib_data_prep.prepare_weather_data?

Signature:
lib_data_prep.prepare_weather_data(
    data: pandas.core.frame.DataFrame,
    drop_unnecessary_columns: bool = True,
) -> pandas.core.frame.DataFrame
Docstring:
Prepare weather data. The following steps are executed:
- Add `has_trace` column
- Convert dates to datetime.

Args:
    data (pd.DataFrame): data from `weather.csv`.
    drop_unnecessary_columns (bool, optional): If true, drops columns that can not be used
    for training/merging to other dataset. Defaults to True.

Returns:
    pd.DataFrame: Dataframe with prepared data.
File:      ~/git/bikehub-model/src/lib_data_prep.py
Type:      function

In [4]:
lib_data_prep.prepare_trip_data?

Signature: lib_data_prep.prepare_trip_data(data: pandas.core.frame.DataFrame) -> pandas.core.frame.DataFrame
Docstring:
Prepare trip data. The following steps are executed:
- Convert trip duration to minutes.
- Remove trips without start station id.
- Build 'is_registered' feature.
- Added average trip time from start station over the 7 days ahead of the trip. Trips longer than
    90 minutes have been ignored.
- Add cyclic variables for day of the week (`dow_sin` and `dow_cos`).
- Drop unnecessary/unusable columns.

Args:
    data (pd.DataFrame): Data from `hubway_trips.csv`.

Returns:
    pd.DataFrame: A dataframe with extra features/preprocessed data. Imputing and encoding
        will still be required (but these should be taken care of in a preprocessing pipeline
        before feeding the data to any ML model).
File:      ~/git/bikehub-model/src/lib_data_prep.py
Type:      function

## Individual dataset preparation

In [5]:
# Preprocess dataset
dw = lib_data_prep.prepare_weather_data(pd.read_csv(PATH_TO_DATA / 'weather.csv'))
dw.head()

,DATE,HPCP,has_trace
0,2011-07-29 09:00:00,0.00,1
1,2011-07-29 10:00:00,0.00,1
2,2011-07-29 18:00:00,0.00,1
3,2011-07-29 21:00:00,0.03,0
4,2011-07-29 22:00:00,0.04,0


In [6]:
dt = lib_data_prep.prepare_trip_data(pd.read_csv(PATH_TO_DATA / 'hubway_trips.csv'))
dt.head()

,seq_id,start_date,strt_statn,end_statn,duration_min,is_registered,avg_duration_prev_7days,dow_sin,dow_cos
0,1,2011-07-28 10:12:00,23.0,23.0,0.150000,1,NaN,0.433884,-0.900969
1,2,2011-07-28 10:21:00,23.0,23.0,3.666667,1,NaN,0.433884,-0.900969
2,3,2011-07-28 10:33:00,23.0,23.0,0.933333,1,NaN,0.433884,-0.900969
3,4,2011-07-28 10:35:00,23.0,23.0,1.066667,1,NaN,0.433884,-0.900969
4,5,2011-07-28 10:37:00,23.0,23.0,0.200000,1,NaN,0.433884,-0.900969


## Merge data

Precipitation data show cumulative figures over the previous hour. 

For the ML model, we will use as feature the HPCP during the previous hour. While this is not the most accurate solution (see note below), it ensures that weather data are available to a deployed model at prediction time. We assume a 1 minute latency for the precipitation data to update. Namely, if a trip started at 12:00, we will assume that the data for the hour ending at 12:00 are not yet available, and
use data for the period 10:00 to 11:00.


> For best accuracy, one may be tempted to to ceil the trip start date to the next hour and join on the weather data
to retrieve the previpitation during the trip. E.g. if trip starts at 14:23, one would want to use weather precipitation data at 15:00, which are the cumulative sum of precipitation from 14 to 15.
>
> The issue with this approach is that:
> 1. At prediction time, we would need to feed a deployed model with real time precipitation data.
> 2. At train time, we are feeding to the model info from the future. E.g., if the trip started at 12:12 but rain only started minutes after, during training we would be feeding this info to the model. As this would not be possible when the model is deployed, the deployed model would perform worse than expected.

As weather data is not available for trips starting after Jul-2013, we also add an indicator of missingness. With this approach, a suitable ML model (which would need to be non-linear) may learn to use weather data, when available, and not to use the when not.  

In [7]:
# We will use as feature the HPCP during the previous hour. This will always be available.
# We assume a 1 minute latency for the precipitation data to update.
# Namely, if a trip started at 12:00, we will assume that the data for the hour ending at 12:00 are
# not yet available, and use data for the period 10:00 to 11:00.
dt['date_to_merge_weather_data'] = (dt['start_date'] - timedelta(minutes=1)).dt.floor('h')
#  dt[['date_to_merge_weather_data', 'start_date']]

df = dt.merge(dw, left_on='date_to_merge_weather_data', right_on='DATE', how='left')

# Add indicator of missingness for period when no weather data were collected.
mask_no_weather_data = (
    (df['date_to_merge_weather_data'] < dw['DATE'].min()) |
    (df['date_to_merge_weather_data'] > dw['DATE'].max())
)
df['has_precip_data'] = 1
df.loc[mask_no_weather_data, 'has_precip_data'] = 0


# Set HPCP and trace to zero when weather data not available
# Note: this can be anything. In theory, a ML model should learn to discard wehether info when the
# 'has_precip_data' is zero.
df.loc[mask_no_weather_data, ['HPCP', 'has_trace']] = 0


# Fill NA
# In the period when whether data were collected, we assume no precipitation whenever HPCP data are
# missing.
df.loc[~mask_no_weather_data, 'HPCP'] = df.loc[~mask_no_weather_data, 'HPCP'].fillna(0.0)
df.loc[~mask_no_weather_data, 'has_trace'] = df.loc[~mask_no_weather_data, 'has_trace'].fillna(0)

# Override types and drop unwanted columns
df['has_trace'] = df['has_trace'].astype(int)
df = df.drop(columns=['date_to_merge_weather_data', 'DATE',])

In [8]:
df.head()

,seq_id,start_date,strt_statn,end_statn,duration_min,is_registered,avg_duration_prev_7days,dow_sin,dow_cos,HPCP,has_trace,has_precip_data
0,1,2011-07-28 10:12:00,23.0,23.0,0.150000,1,NaN,0.433884,-0.900969,0.0,0,0
1,2,2011-07-28 10:21:00,23.0,23.0,3.666667,1,NaN,0.433884,-0.900969,0.0,0,0
2,3,2011-07-28 10:33:00,23.0,23.0,0.933333,1,NaN,0.433884,-0.900969,0.0,0,0
3,4,2011-07-28 10:35:00,23.0,23.0,1.066667,1,NaN,0.433884,-0.900969,0.0,0,0
4,5,2011-07-28 10:37:00,23.0,23.0,0.200000,1,NaN,0.433884,-0.900969,0.0,0,0


In [9]:
df.to_csv(PATH_TO_DATA / 'training-data.csv', index=False)